In [1]:
import pandas as pd
import numpy as np

# Load data
df = pd.read_csv('final.csv')

# Basic cleaning
def clean_data(df):
    # Handle missing SEO targets
    df['has_seo'] = ~df['seo_title'].isna()
    
    # Clean text columns
    text_cols = ['product_title', 'product_description', 'product_features']
    for col in text_cols:
        df[col] = df[col].str.replace('\r\n', ' ').str.strip()
    
    # Convert features to lists
    df['product_features'] = df['product_features'].str.split('|')
    
    return df

cleaned_df = clean_data(df)

In [2]:
# Check SEO coverage
print(f"{cleaned_df['has_seo'].mean():.1%} rows have SEO content")

# Category distribution
print(cleaned_df['product_category'].value_counts())

# Example row inspection
sample = cleaned_df.iloc[0]
print(f"""
Input Title: {sample['product_title']}
→ SEO Title: {sample['seo_title']}
Features: {sample['product_features'][:3]}...
""")

100.0% rows have SEO content
product_category
Electronics       1286
Home & Kitchen    1252
Fashion           1238
Beauty            1224
Name: count, dtype: int64

Input Title: Wheeler-Hartman Cause 188
→ SEO Title: Wheeler-Hartman Cause 188 - Stretch Fit & Organic Cotton | Fashion
Features: ['Stretch Fit, Organic Cotton, UV Protection, Breathable']...



In [3]:
# Create prompt-completion pairs
training_data = []
for _, row in cleaned_df[cleaned_df['has_seo']].iterrows():
    prompt = f"""
    Generate SEO content for this product:
    Title: {row['product_title']}
    Description: {row['product_description'][:200]}...
    Features: {', '.join(row['product_features'][:5])}
    Category: {row['product_category']}
    """
    
    completion = f"""
    SEO Title: {row['seo_title']}
    Keywords: {row['seo_keywords']}
    Meta Description: {row['seo_meta_polished']}
    """
    
    training_data.append({'prompt': prompt, 'completion': completion})

# Convert to DataFrame
train_df = pd.DataFrame(training_data)
train_df.to_csv('seo_training_data.csv', index=False)

In [10]:
# First install the package (uncomment if needed)
# !pip install openai pandas

import pandas as pd


# Sample DataFrame creation (replace with your actual data)
train_df = pd.DataFrame({
    'prompt': ["Generate SEO for wireless earbuds"],
    'completion': ["SEO Title: Premium Wireless Earbuds with 24hr Battery"]
})

# Save to JSONL
train_df[['prompt', 'completion']].to_json(
    'seo_data.jsonl',
    orient='records',
    lines=True
)

print("File created successfully!")

# Use OpenAI CLI (run in terminal):
# openai api fine_tunes.create -t seo_data.jsonl -m curie

File created successfully!


ModuleNotFoundError: No module named 'openai'

In [23]:
!pip install openai

In [29]:
openai api fine_tuning.jobs.create \
  -t seo_data.jsonl \
  -m gpt-3.5-turbo \  # or another supported model
  --suffix "seo-generator"  # Optional: adds identifier to your model name

SyntaxError: invalid syntax (2604792437.py, line 1)

In [25]:
!pip install transformers torch sentencepiece

  Using cached transformers-4.53.1-py3-none-any.whl.metadata (40 kB)
  Using cached sentencepiece-0.2.0.tar.gz (2.6 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [48 lines of output]
      Traceback (most recent call last):
        File "C:\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
          ~~~~^^
        File "C:\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Python313\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 143, in get_requires_for_build_wheel
          return hook(config_settings)
        File "C:\Users\archu\AppData\Local\Temp\pip-build-env-8i_jxj0y\overlay\Lib\site-packages\setuptools\build_meta.py", line 331, in get_requires_for_build_wheel
          return self._get_build_requires(config_sett

In [27]:
from transformers import pipeline

# Use a smaller model
seo_pipeline = pipeline(
    'text-generation',
    model='distilgpt2',  # Smaller version
    device='cpu'
)

c:\Users\archu\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\archu\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\archu\.cache\huggingface\hub\models--distilgpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator.

ValueError: Could not load model distilgpt2 with any of the following classes: (<class 'transformers.models.auto.modeling_auto.AutoModelForCausalLM'>, <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>). See the original errors:

while loading with AutoModelForCausalLM, an error is thrown:
Traceback (most recent call last):
  File "c:\Users\archu\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\response.py", line 748, in _error_catcher
    yield
  File "c:\Users\archu\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\response.py", line 873, in _raw_read
    data = self._fp_read(amt, read1=read1) if not fp_closed else b""
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\archu\AppData\Local\Programs\Python\Python312\Lib\site-packages\urllib3\response.py", line 856, in _fp_read
    return self._fp.read(amt) if amt is not None else self._fp.read()
           ^^^^^^^^^^^^^^^^^^
  File "c:\Users\archu\AppData\Local\Programs\Python\Pyt

In [30]:
from transformers import pipeline

# Small model for beginners
seo_pipeline = pipeline(
    'text-generation',
    model='gpt2',
    device='cpu'  # Change to 'cuda' if you have GPU
)

# Test with sample prompt
sample_prompt = training_data[0]['prompt']
generated = seo_pipeline(sample_prompt, max_length=200)
print(generated[0]['generated_text'])

c:\Users\archu\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\archu\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP 


    Generate SEO content for this product:
    Title: Wheeler-Hartman Cause 188
    Description: Case quite start culture fight. Seat community process....
    Features: Stretch Fit, Organic Cotton, UV Protection, Breathable
    Category: Fashion
     Product Type: Fashions and Accessories
      Email: mh@hqr.com
      Description: Case quite start culture fight. Seat community process....
   Features: Stretch Fit, Organic Cotton, UV Protection, Breathable
     Product Type: Fashions and Accessories
      Email: mh@hqr.com
